In [ ]:
!pip install pgmpy

In [ ]:
from pgmpy.models import DiscreteBayesianNetwork as DBN
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
model_structure = DBN([('Burglary', 'Alarm'),
                       ('Earthquake', 'Alarm'),
                       ('Alarm', 'JohnCalls'),
                       ('Alarm', 'MerryCalls')])

In [ ]:
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2,
                          values=[[0.999], [0.001]])
# row 1 is P(Burglary=0)
# row 2 is P(Burglary=1)
print(cpd_burglary)

+-------------+-------+
| Burglary(0) | 0.999 |
+-------------+-------+
| Burglary(1) | 0.001 |
+-------------+-------+


In [ ]:
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2,
                          values=[[0.998], [0.002]])
print(cpd_earthquake)

+---------------+-------+
| Earthquake(0) | 0.998 |
+---------------+-------+
| Earthquake(1) | 0.002 |
+---------------+-------+


In [ ]:
cpd_alarm = TabularCPD(variable = 'Alarm', variable_card=2, evidence=['Burglary', 'Earthquake'],
                       evidence_card=[2,2],
                       values = [[0.999, 0.71, 0.06, 0.05],[0.001,0.29, 0.94, 0.95]])
print(cpd_alarm)

+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+---------------+


In [ ]:
cpd_johncalls = TabularCPD(variable='JohnCalls', variable_card=2,evidence=['Alarm'], evidence_card=[2],
                          values=[[0.95,0.1],[0.05,0.9]])
print(cpd_johncalls)

+--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| JohnCalls(0) | 0.95     | 0.1      |
+--------------+----------+----------+
| JohnCalls(1) | 0.05     | 0.9      |
+--------------+----------+----------+


In [ ]:
cpd_merrycalls = TabularCPD(variable='MerryCalls', variable_card=2, evidence=['Alarm'], evidence_card=[2],
                          values=[[0.99, 0.30],[0.01, 0.70]])
print(cpd_merrycalls)

+---------------+----------+----------+
| Alarm         | Alarm(0) | Alarm(1) |
+---------------+----------+----------+
| MerryCalls(0) | 0.99     | 0.3      |
+---------------+----------+----------+
| MerryCalls(1) | 0.01     | 0.7      |
+---------------+----------+----------+


In [ ]:
model_structure.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_merrycalls)
model_structure.check_model()

True

In [ ]:
inference = VariableElimination(model_structure)

# Query 1

In [ ]:
print('Alarm rings. John calls, Merry Calls given burgulary and Earthquake doesnt happen')
evidence = {
    'Burglary': 0,
    'Earthquake': 0
}
query_result = inference.query(variables=['Alarm','JohnCalls','MerryCalls'], evidence=evidence)
print(query_result)

Alarm rings. John calls, Merry Calls
+----------+--------------+---------------+-----------------------------------+
| Alarm    | JohnCalls    | MerryCalls    |   phi(Alarm,JohnCalls,MerryCalls) |
+==========+==============+===============+===================================+
| Alarm(0) | JohnCalls(0) | MerryCalls(0) |                            0.9396 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(0) | JohnCalls(0) | MerryCalls(1) |                            0.0095 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(0) | JohnCalls(1) | MerryCalls(0) |                            0.0495 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(0) | JohnCalls(1) | MerryCalls(1) |                            0.0005 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(1) | JohnCalls(0) | MerryCalls(0) |                            0.0000 |
+--

In [ ]:
values = query_result.values[1][1][1]
print(values)

0.0006300000000000001


# Query 2

In [ ]:
print('John Calls')
query_result = inference.query(variables=['JohnCalls'])
print(query_result)


John Calls
+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.9479 |
+--------------+------------------+
| JohnCalls(1) |           0.0521 |
+--------------+------------------+


In [ ]:
values = query_result.values[1]
print(values)

0.284171835364393


# Query 3

In [ ]:
print('John and Merry both call provided Burgulary happens')
evidence = {'JohnCalls':1, 'MerryCalls':1}
query_result = inference.query(variables=['Burglary'], evidence = evidence)
print(query_result)

John and Merry both call provided Burgulary happens
+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
values = query_result.values[1]
print(values)

0.284171835364393


# Query 4

In [ ]:
print('Alarm rings but both John and Merry dont call')
query_result = inference.query(variables=['Alarm'], evidence = {'JohnCalls':0, 'MerryCalls':0})
print(query_result)


Alarm rings but both John and Merry dont call
+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.9999 |
+----------+--------------+
| Alarm(1) |       0.0001 |
+----------+--------------+


In [ ]:
values = query_result.values[1]
print(values)

8.046530968883623e-05


# Query 5

In [ ]:
print('Merry Calls')
query_result = inference.query(variables=['MerryCalls'])
print(query_result)


Merry Calls
+---------------+-------------------+
| MerryCalls    |   phi(MerryCalls) |
+===============+===================+
| MerryCalls(0) |            0.9883 |
+---------------+-------------------+
| MerryCalls(1) |            0.0117 |
+---------------+-------------------+


In [ ]:
values = query_result.values[1]
print(values)

0.01173634498


# Query 6

In [ ]:
print('Alarm doesnt rings,  John doesnt Calls , MerryCalls given Burgulary doesnt happen, Earthquake does')
evidence = {
    'Burglary': 0,
    'Earthquake': 1
}
query_result = inference.query(variables=['Alarm','JohnCalls','MerryCalls'], evidence=evidence)
print(query_result)

+----------+--------------+---------------+-----------------------------------+
| Alarm    | JohnCalls    | MerryCalls    |   phi(Alarm,JohnCalls,MerryCalls) |
+==========+==============+===============+===================================+
| Alarm(0) | JohnCalls(0) | MerryCalls(0) |                            0.6678 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(0) | JohnCalls(0) | MerryCalls(1) |                            0.0067 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(0) | JohnCalls(1) | MerryCalls(0) |                            0.0351 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(0) | JohnCalls(1) | MerryCalls(1) |                            0.0004 |
+----------+--------------+---------------+-----------------------------------+
| Alarm(1) | JohnCalls(0) | MerryCalls(0) |                            0.0087 |
+----------+--------------+-------------

In [ ]:
values = query_result.values[0][0][1]
print(values)

0.006745
